## Import pakages.

In [1]:
import re
import csv
import json
import neo4j
import py2neo
from pprint import pprint
from py2neo import Database
from py2neo import Graph, Node, Relationship, NodeMatcher

## Data preparation.

In [2]:
def get_val(data:dict,key:str) -> str:
    
    ''' Get values from dictionary, 
        if key not in dictionary return None '''
    
    if key not in data.keys():
        return None
    elif data[key]=='':
        return None
    else:
        return data[key]

### Acquisition for all companies.

Created a csv file with company acquisition, year and amount.

In [3]:
# Dictiionary for acquisition. {company:{acquired_company:[year, amount]}...}
acquisition={}
with open('./info_box/Acquisition.csv') as Acquisition:
    acquisition_all = csv.DictReader(Acquisition)
    for row in acquisition_all:
        if row['Company'] in acquisition.keys():
            acquisition[row['Company']][row['Acquisition']]= [get_val(row,'Year'),get_val(row,'Amount')]
        else:
            acquisition[row['Company']]={}
            acquisition[row['Company']][row['Acquisition']] = [get_val(row,'Year'),get_val(row,'Amount')]
        

In [4]:
pprint(acquisition)

{'Alibaba Group': {'Alibaba Pictures': ['2014', ' $805 million'],
                   'Vendio': ['2010', None],
                   'Youku': ['2015', None]},
 'Walmart': {'Asda': [None, None],
             'Jet.com': ['2016', None],
             'Kmart': ['1993', None],
             'Moosejaw': ['2017', '$51 million'],
             'Vudu': ['2010', '$100 million']}}


### NAICS Code: Create company_naics_code dictionary.

Note: Add company name to the company list in "scrape_naics.py".

- Youtube ==> Youku "can not find company" ;

- Paramount Pictures ==> Alibaba Pictures "can not find company" 

company_list = ['Alibaba', 'Walmart', 'eBay', 'Amazon','Youtube','Paramount Pictures','Costco','Kroger','Target','Vudu','Jet.com','Kmart']

In [5]:
# Unique naics codes.
naics=dict() 
# Company and 6 digits Naics Code, which we need create relation between code and company. 
company_code=dict() 

with open('./naics_scrape/naics_scraped.csv') as file:
    data = csv.DictReader(file)
    for row in data:
        company = row['Company']
        code = row['NAICS_CODE']
        title = row['NAICS_TITLE']
        naics[code] = title
        if company not in company_code.keys():
             company_code[company]=code
                
# Change 'Youtube' and 'Paramount Pictures' to 'Youku' and 'Alibaba Pictures'.
company_code['Youku'] = company_code['Youtube']
company_code['Alibaba Pictures'] = company_code['Paramount Pictures']
company_code['Alibaba Group'] = company_code['Alibaba']
del company_code['Youtube']
del company_code['Paramount Pictures']
del company_code['Alibaba']

# Extract unique root code.
root_code = [code for code in naics.keys() if len(code)==2 or re.match('\d+-\d+',code)]

# Code with 6 digits.
code6 = [code for code in naics.keys() if len(code)==6]
# Code with 5 digits.
code5 = [code for code in naics.keys() if len(code)==5 and not re.match('\d+-\d+',code)]
# Code with 4 digits.
code4 = [code for code in naics.keys() if len(code)==4]
# Code with 3 digits.
code3 = [code for code in naics.keys() if len(code)==3]


In [6]:
print(root_code)
print(code3)
print(code4)
print(code5)
print(code6,'\n')
pprint(company_code)

['56', '44-45', '42', '51', '31-33', '23']
['561', '445', '443', '423', '512', '446', '336', '518', '237']
['5619', '4451', '4431', '4237', '5121', '4461', '3364', '5182', '2379']
['56199', '44511', '44314', '42372', '51219', '44611', '33641', '51821', '23799']
['561990', '445110', '443142', '423720', '512191', '446110', '336413', '518210', '237990'] 

{'Alibaba Group': '561990',
 'Alibaba Pictures': '512191',
 'Amazon': '443142',
 'Costco': '446110',
 'Jet.com': '237990',
 'Kmart': '445110',
 'Kroger': '445110',
 'Target': '336413',
 'Vudu': '518210',
 'Walmart': '445110',
 'Youku': '423720',
 'eBay': '561990'}


In [7]:
naics

{'561990': 'All Other Support Services',
 '56199': 'All Other Support Services',
 '5619': 'Other Support Services',
 '561': 'Administrative and Support Services',
 '56': 'Administrative and Support and Waste Management and Remediation Services',
 '445110': 'Supermarkets and Other Grocery (except Convenience) Stores',
 '44511': 'Supermarkets and Other Grocery (except Convenience) Stores',
 '4451': 'Grocery Stores',
 '445': 'Food and Beverage Stores',
 '44-45': 'Retail Trade',
 '443142': 'Electronics Stores',
 '44314': 'Electronics and Appliance Stores',
 '4431': 'Electronics and Appliance Stores',
 '443': 'Electronics and Appliance Stores',
 '423720': 'Plumbing and Heating Equipment and Supplies (Hydronics) Merchant Wholesalers',
 '42372': 'Plumbing and Heating Equipment and Supplies (Hydronics) Merchant Wholesalers',
 '4237': 'Hardware, and Plumbing and Heating Equipment and Supplies Merchant Wholesalers',
 '423': 'Merchant Wholesalers, Durable Goods',
 '42': 'Wholesale Trade',
 '51219

## Create graph. 
### Company Nodes and relationships

**NOTE** : I used the script in `'./scrapy_spider/scrapy_spider/spiders/wiki_spider_info.py'` to extracted the info box and saved in `info_box` folder.

In [8]:
def Create_Company_Node(json):
    
    '''Create company node'''
    
    company_node = Node('Company', 
                    name=get_val(json,'Title'),
                    organization_name=get_val(json,'Organization_name'),
                    founded=get_val(json,'Founded'),
                    founder=get_val(json,'Founder'),
                    industry=get_val(json,'industry'), 
                    products=get_val(json,'Products'), 
                    services=get_val(json,'Services'), 
                    revenue=get_val(json,'Revenue'), 
                    employees=get_val(json,'Number of employees'),
                    website=get_val(json,'Website'))
    
    company_node.__primarylabel__ = "Company"
    company_node.__primarykey__ = "name"
    
    return company_node

In [10]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234') 
tx = g.begin()

# Main company_Alibaba.
with open('./info_box/main_company.json') as main_company:
    data = json.load(main_company)
    for d in data:
        Alibaba = Create_Company_Node(d)         
        tx.create(Alibaba)

# Competitor_Alibaba. 
# ** Please chage the Amazon title in Competitor_Alibaba.json': "Amazon (Company)" --> "Amazon" ** 
with open('./info_box/Competitor_Alibaba.json') as competitors:
    data = json.load(competitors)
    for d in data:
        company_node = Create_Company_Node(d)
        relation = Relationship(Alibaba,'Competitor',company_node)
        tx.create(relation)
        
# Acquisition_Alibaba.
with open('./info_box/Acquisition_Alibaba.json') as acquisitions:
    data = json.load(acquisitions)
    for d in data:
        company_name = d['Title']
        company_node = Create_Company_Node(d)
        Y = acquisition['Alibaba Group'][company_name][0]
        A = acquisition['Alibaba Group'][company_name][1]
        relation = Relationship(Alibaba,'Acquired',company_node,year=Y,amount=A)
        tx.create(relation)        

tx.commit()

In [11]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
matcher = NodeMatcher(g)
tx = g.begin()

# Find Walmart node.
Walmart = matcher.match('Company',name='Walmart').first()

# Competitor_Walmart.
with open('./info_box/Competitor_Walmart.json') as competitors:
    data = json.load(competitors)
    for d in data:
        company_node = Create_Company_Node(d)
        relation = Relationship(Walmart,'Competitor',company_node)
        tx.create(relation)

# Acquisition_Walmart. 
with open('./info_box/Acquisition_Walmart.json') as acquisitions:
    data = json.load(acquisitions)
    for d in data:
        company_name = d['Title']
        company_node = Create_Company_Node(d)
        Y = acquisition['Walmart'][company_name][0]
        A = acquisition['Walmart'][company_name][1]
        relation = Relationship(Walmart,'Acquired',company_node,year=Y,amount=A)
        tx.create(relation)
        
tx.commit()

### NAICS code nodes and relationships.

In [12]:
def Create_Code_Node(code):
    
    '''Create NAICS Ccode node'''
    
    code_node = Node('NAICS', code=code, title=naics[code])
    
    code_node.__primarylabel__ = "NAICS"
    code_node.__primarykey__ = "code"
    
    return code_node
    

In [13]:
def Check_Code_Node(code):
    
    '''Check if NAICS Ccode node already exists'''
    
    node = matcher.match('NAICS', code=code)
    
    if node == None:
        new = Create_Code_Node(code)
        return new
    else:
        return node

#### Root Code. 

In [14]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
tx = g.begin()

# Create Root Code Node. (2-digits or d-d)
for root in root_code:
    root_node = Create_Code_Node(root)
    tx.create(root_node)
    
tx.commit()

#### 3-digit-code

In [15]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
tx = g.begin()
matcher = NodeMatcher(g)

# Create 3-digit-code Node.
for code in code3:
    node3 = Create_Code_Node(code)
    
    # Find the root code node.
    if code[:-1] in ['44','45']:
        node2 = matcher.match('NAICS', code='44-45').first()    
    elif code[:-1] in ['31','33']:
        node2 = matcher.match('NAICS', code='31-33').first()
    else:
        node2 = matcher.match('NAICS', code=code[:-1]).first()
        
    tx.create(Relationship(node3,'SubClassOf',node2))

tx.commit()

#### 4-digit-code

In [16]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
tx = g.begin()
matcher = NodeMatcher(g)

# Create 4-digit-code Node.
for code in code4:
    node4 = Create_Code_Node(code)
    node3 = matcher.match('NAICS', code=code[:-1]).first()
    tx.create(Relationship(node4,'SubClassOf',node3))

tx.commit()

#### 5-digit-code

In [17]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
tx = g.begin()
matcher = NodeMatcher(g)

# Create 5-digit-code Node.
for code in code5:
    node5 = Create_Code_Node(code)
    node4 = matcher.match('NAICS', code=code[:-1]).first()
    tx.create(Relationship(node5,'SubClassOf',node4))

tx.commit()

#### 6-digit-code

In [18]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
tx = g.begin()
matcher = NodeMatcher(g)

# Create 5-digit-code Node.
for code in code6:
    node6 = Create_Code_Node(code)
    node5 = matcher.match('NAICS', code=code[:-1]).first()
    tx.create(Relationship(node6,'SubClassOf',node5))

tx.commit()

#### Create relationships between company and 6-digit-code.

In [19]:
# Connect database.(set your own password)
g = Graph(host='localhost',user='neo4j',password='1234')
tx = g.begin()

# Create relationships.
matcher = NodeMatcher(g)
for company in company_code:
    CompanyNode = matcher.match('Company', name=company).first()
    CodeNode = matcher.match('NAICS', code=company_code[company]).first()
    tx.create(Relationship(CodeNode,'COMPANY',CompanyNode))
    
tx.commit()